## openpyxl 과 excel file pivot table

In [8]:
import pandas as pnds
import pickle
import datetime

# dataframe으로 저장된 opera trial balance 읽기
work_date = datetime.datetime.now() - datetime.timedelta(10) # 20220713
target_date = work_date.strftime("%Y%m%d")                  # 어제 날짜 포맷

dfdata_dir = f'./data/{target_date}/dfdata/'                # 읽어들일 dfdata디렉토리 './data/YYYYMMDD/dfdata/'
df_filename = 'df_kicc_history_' + target_date               # 파일 이름: df_kicc_history_YYYYMMDD

#   Datafarame loading
try:
    with open(dfdata_dir + df_filename, "rb") as file:
        history_df = pickle.load(file)
except Exception as e:
    with open('./error.log', 'a') as file:
        file.write(
            f'[edi_opera.py - Reading Data] <{datetime.datetime.now()}> pickle file-reading error ({df_filename}) ===> {e}\n'
        )
    raise(e)


history_df

,거래고유번호,승인구분,date,카드번호,발급카드사,매입카드사,금액,할부개월,승인번호
21,130733848107,승인,2022-07-13 07:55:06,5288-1500-0153-1305,현대마스터개인,현대카드,11000,일시불,00260466
20,131111723337,승인,2022-07-13 11:36:53,5557-9774-5964-6142,해외마스타,하나구외환,50742,일시불,003000
19,131121864954,승인,2022-07-13 11:53:29,5587-1725-6984-8564,해외마스타,하나구외환,53249,일시불,746448
18,131212383607,승인,2022-07-13 12:45:40,3779-8973-0830-006,삼성아멕스,삼성카드,88825,일시불,28098696
17,131433079480,승인,2022-07-13 14:42:21,5365-1005-0627-8379,카카오뱅크체크,국민카드,206399,일시불,25103548
16,131522632248,승인,2022-07-13 15:30:20,9440-1244-4258-0863,NH농협비씨체크,비씨카드,66330,일시불,45554823
15,131633368709,승인,2022-07-13 16:14:43,5570-4204-8815-0052,KB국민카드,국민카드,35000,일시불,30003456
14,131611955355,승인,2022-07-13 16:15:26,9470-1136-3085-7988,NH농협비씨카드,비씨카드,60000,일시불,45840168
13,131611957804,승인,2022-07-13 16:15:50,5181-8500-1059-4602,하나카드,하나구외환,78166,일시불,13087910
12,131612104208,승인,2022-07-13 16:35:55,5570-4204-8815-0052,KB국민카드,국민카드,11000,일시불,30003467


In [22]:
import openpyxl
# 1. EDI 템플릿 엑셀 파일을 로딩
history_excel_filename = './신용거래내역조회_tmp.xlsx'                                      # '신용거래내역' 템플릿 파일

# 수식 포함하여 템플릿 엑셀 파일 로딩
try:
    edi_excel = openpyxl.load_workbook(history_excel_filename, data_only=False)
    
    pivot_sheet = edi_excel['pivot']        # Open the specified Sheet
    # 아래 2라인을 하지 않는다면, 좀 더 빨리 로딩해서 수동으로 '피벗테이블분석-데이터원본고침'(or '화면고침')을 해도 된다
    # pivot = pivot_sheet._pivots[0]          # Any one can share the same cache
    # pivot.cache.refreshOnLoad = True        # Refresh loading
    
    # 자동 refresh를 하지 앟는 경우에는, 경고의 의미로 화면 내에 "'테이터원본고침' 실시 필요" 라는 문구를 기입한다
    pivot_sheet['A2'].value = '피벗테이블분석-테이터원본변경 필요함'
    pivot_sheet['A2'].style = 'Title'
    
    # data sheet
    data_ws = edi_excel['data']
except Exception as e:
    with open('./error.log', 'a') as file:
        file.write(
            f'[edi_opera.py - Reading Data] <{datetime.datetime.now()}> openpyxl file-reading error ({history_excel_filename}) ===> {e}\n'
        )
    raise(e)

# 1행부터 마지막행까지 원본 데이터 삭제
print(data_ws.max_row, data_ws.max_column)
data_ws.delete_rows(1, data_ws.max_row)
print(data_ws.max_row, data_ws.max_column)

from openpyxl.utils.dataframe import dataframe_to_rows

# 컬럼 이름부터 시작하여 각 행을 엑셀에 추가
for row in dataframe_to_rows(history_df, index=False, header=True):     # index는 제외해야함
    data_ws.append(row)

edi_excel.save('./mytest.xlsx')

47 9
1 1
